In [1]:
import numpy as np
import smash_text as smt

# OPTIONS
skip_pools = 0 # set to true to use only bracket data


# EXTRACT DATA FROM TABLES
''' First get number of datapoints and number of player names '''
    
file = open( 'smashdata.csv' )
numitems = 0
playernames = {}
charnames = {}
game_outcomes = []
tournaments = {}
tourney_years = {}
for line in file:
    line = line.strip()
    parts = line.split(',')
    if parts[7] == 'pools':
        if skip_pools == 1:
            continue
    tourney_years[ parts[0] ] = 1
    tournaments[ parts[1] ] = 1
    numitems = numitems + 1
    playernames[ parts[2] ] = 1
    playernames[ parts[3] ] = 1
    charnames[ parts[4] ] = 1
    charnames[ parts[5] ] = 1
    game_outcomes.append(parts[6])

numchars = len(charnames)
numnames = len(playernames)

smt.print_header( numnames, numitems, tournaments, tourney_years, skip_pools )


Smash 64 results from recorded games at the following tournaments:
		beast7
		goml
		genesis 3
		ssc
		snosa2
		pound

During these years:
		2016
		2017
Includes bracket and pools
Number of players in data: 158
Number of games in data: 765


In [2]:
# Next retrieve character-based results

''' Next construct hastables to map from name to number '''
number2name = {}
name2number = {}
whichname = 0
for item in playernames:
    name2number[ item ] = whichname
    number2name[ whichname ] = item
    whichname = whichname + 1


''' Now construct hashtables for characters '''
number2char = {}
char2number = {}
whichchar = 0
for item in charnames:
    char2number[ item ] = whichchar
    number2char[ whichchar ] = item
    whichchar = whichchar + 1

''' Construct adjacency matrices for game stats ??? '''
# GameMat gives number of games played between players.
# CharGameMat[x,y] gives number of games played between characters x and y
# CharOutcomeMat[x,y] gives number of wins of char y over char x
GameMat = np.zeros((numnames, numnames))
CharGameMat = np.zeros((numchars, numchars))
CharOutcomeMat = np.zeros((numchars, numchars))
pool_or_bracket = []

file.close()
file = open( 'smashdata.csv' )
line_number = 0

char_wins = np.zeros((numchars,1))
for line in file:
    line = line.strip()
    parts = line.split(',')
    pool_or_bracket.append(parts[-1])
    if parts[7] == 'pools':
        if skip_pools == 1:
            continue
    xj = name2number[ parts[2] ]
    yj = name2number[ parts[3] ]
    GameMat[ xj, yj ] = GameMat[ xj, yj ] + 1
    GameMat[ yj, xj ] = GameMat[ yj, xj ] + 1
    xchar = char2number[ parts[4] ]
    ychar = char2number[ parts[5] ]
    winchar = xchar
    losechar = ychar
    if int(parts[6]) == 2:
        winchar = ychar
        losechar = xchar
    #if int(parts[6]) == 1:
    CharOutcomeMat[ losechar, winchar ] = CharOutcomeMat[ losechar, winchar ] + 1
    if winchar != losechar:
        char_wins[ winchar, 0 ] = char_wins[ winchar, 0 ] + 1
    
    CharGameMat[ xchar, ychar ] = CharGameMat[ xchar, ychar ] + 1
    CharGameMat[ ychar, xchar ] = CharGameMat[ ychar, xchar ] + 1
    line_number = line_number + 1

'''
These data structures contain actual tournament game/match data:
    GameMat
    CharGameMat
    
These contain maps from data structures indices to player/character names:
    char2number
    number2char
    name2number
    number2name
    
'''
char1 = []
char2 = []
char_tots = []
outcome_prct = []
num_matches = []
num_outcomes = []
for i in range( len(char2number) ) :
    for j in range( i, len(char2number) ) :
        char1.append(number2char[i])
        char2.append(number2char[j])
        num_matches.append(CharGameMat[ i,j ])
        num_outcomes.append(CharOutcomeMat[ j,i ])
        if CharGameMat[i,j] != 0:
            outcome_prct.append(CharOutcomeMat[j,i]/CharGameMat[ i,j ])
        if CharGameMat[i,j] == 0:
            outcome_prct.append(-5)

for i in range(len(char2number)):
    char_tots.append( [number2char[i],0] )
    for j in range( len(char2number) ):
        char_tots[i][1] = char_tots[i][1] + CharGameMat[ i,j ]

        
# NOW PRINT
print("Matchup percentages \n")
print("char1\tchar2 \t# games \tchar1 wins \tprct win by char 1")
inds = list(reversed(np.argsort(num_matches)))
for i in inds:
    if num_matches[i] < 1:
        continue
    print( char1[i] + '\t' + char2[i] + '\t' + str(num_matches[ i ] ) + '\t\t' + str(num_outcomes[i]) + '\t\t' + ("%.2f" % outcome_prct[i]) )

char_tots
charnames = []
charnums = []
for j in range(len(char_tots)):
    charnames.append(char_tots[j][0])
    charnums.append(char_tots[j][1])

inds = list(reversed(np.argsort(charnums)))
sorted_data = []
for j in inds:
    sorted_data.append(char_tots[j])

# This time without dittos
char_tots
charnames = []
charnums = []
charwins = char_wins
for j in range(len(char_tots)):
    charnames.append(char_tots[j][0])
    charnums.append(char_tots[j][1] - CharGameMat[j,j])

inds = list(reversed(np.argsort(charnums)))

print("\nCharacter-based wins in recorded games\n(excludes dittos)")
print("char\t\t# games\t\t% wins")
for i in inds:
    #    print( charnames[i] + '\t' + str(charnums[ i ] ) + '\t' + str(charwins[i]) + ("%.2f" % (charwins[i][0]/charnums[i]) ) )
    print( charnames[i] + '\t\t' + str(charnums[ i ] ) + '\t\t' + ("%.2f" % (charwins[i][0]/charnums[i]) ) )
    
# Print just character appearances
# (including dittos)
charappearances = sum(CharGameMat)
inds = list(reversed(np.argsort(charappearances)))
print("\nTotal games " + str(sum(sum(CharGameMat))/2 ) + " (two characters appear per game)")
print("char\t\t# appearances" + '\t' + "% of total")
for i in inds:
    #print( charnames[i] + '\t\t' + str(sum(CharGameMat[ i, : ])) + '\t\t' + ("%.2f" % (sum(CharGameMat[ i, : ])/(sum(sum(CharGameMat))) ) ) )
    print( charnames[i] + '\t\t' + str(charappearances[ i ]) + '\t\t' + ("%.2f" % (charappearances[i]/(sum(sum(CharGameMat))) ) ) )


smt.print_header( numnames, numitems, tournaments, tourney_years, skip_pools )

Matchup percentages 

char1	char2 	# games 	char1 wins 	prct win by char 1
pika	pika	142.0		71.0		0.50
falcon	pika	86.0		38.0		0.44
pika	kirby	73.0		44.0		0.60
yoshi	pika	73.0		33.0		0.45
falcon	falcon	42.0		21.0		0.50
pika	puff	41.0		20.0		0.49
fox	pika	40.0		13.0		0.33
yoshi	falcon	36.0		22.0		0.61
yoshi	kirby	32.0		13.0		0.41
fox	fox	28.0		14.0		0.50
fox	falcon	26.0		12.0		0.46
falcon	kirby	23.0		10.0		0.43
link	kirby	16.0		9.0		0.56
falcon	link	14.0		9.0		0.64
puff	kirby	13.0		4.0		0.31
dk	dk	12.0		6.0		0.50
falcon	puff	12.0		8.0		0.67
falcon	ness	12.0		8.0		0.67
mario	pika	12.0		6.0		0.50
yoshi	puff	11.0		6.0		0.55
yoshi	yoshi	10.0		5.0		0.50
fox	kirby	10.0		6.0		0.60
yoshi	fox	10.0		5.0		0.50
yoshi	mario	8.0		2.0		0.25
pika	dk	7.0		5.0		0.71
pika	link	7.0		4.0		0.57
falcon	samus	7.0		4.0		0.57
falcon	mario	6.0		6.0		1.00
mario	puff	6.0		1.0		0.17
yoshi	ness	6.0		5.0		0.83
yoshi	link	6.0		5.0		0.83
fox	puff	6.0		3.0		0.50
kirby	kirby	6.0		3.0		0.50
fox	dk	4.0		0.0		0.00
luigi	mari

In [3]:
# GET PLAYER DATA
''' Get player game totals, and char totals '''
    
file.close()
file = open( 'smashdata.csv' )
numitems = 0
playerchargames = {}
playercharwins = {}
for line in file:
    line = line.strip()
    parts = line.split(',')
    if parts[-1] == 'pools':
        if skip_pools == 1:
            continue
    gameplayernames = [ parts[2], parts[3] ]
    gamecharnames = [ parts[4], parts[5] ]
    for j in [0,1]:
        current_playerchar = gameplayernames[j] + '-' + gamecharnames[j]
        playerchargames[ current_playerchar ] = 0
        playercharwins[ current_playerchar ] = 0
file.close()


file = open( 'smashdata.csv' )
for line in file:
    line = line.strip()
    parts = line.split(',')
    if parts[-1] == 'pools':
        if skip_pools == 1:
            continue
    gameplayernames = [ parts[2], parts[3] ]
    gamecharnames = [ parts[4], parts[5] ]
    for j in [0,1]:
        current_playerchar = gameplayernames[j] + '-' + gamecharnames[j]
        playerchargames[ current_playerchar ] = playerchargames[ current_playerchar ] + 1
    whichwin = int(parts[6]) - 1
    winningplayer = gameplayernames[whichwin] + '-' + gamecharnames[whichwin]
    playercharwins[winningplayer] = playercharwins[winningplayer] + 1
file.close()

# Re-organize to sort by wins
playchar2num = {}
num2playchar = []
num2wins = []
numitems = 0
for key, val in playerchargames.items():
    playchar2num[key] = numitems
    num2playchar.append(key)
    num2wins.append(val)
    numitems = numitems + 1
    
inds = list(reversed(np.argsort(num2wins)))
print("\nPlayer-character game data")
print("\nplayer-character" + '\t\t' + "# games" + '\t\t' + "# wins" + '\t\t' + "% win")
for j in range(len(playchar2num)):
    idx = inds[j]
    key = num2playchar[idx]
    val = int(playerchargames[key])
    valwins = int(playercharwins[key])
    print( key.ljust(20) + '\t\t' + str(val) + '\t\t' + str(valwins) + '\t\t' +  ("%.2f" % (valwins/val) ) )

    


Player-character game data

player-character		# games		# wins		% win
revan-kirby         		65		36		0.55
kerokeroppi-pika    		57		33		0.58
barksanchez-pika    		53		27		0.51
mariguas-pika       		51		33		0.65
wizzrobe-yoshi      		43		21		0.49
wangera-puff        		41		28		0.68
tacos-yoshi         		35		22		0.63
superboomfan-falcon 		35		26		0.74
thez-pika           		34		18		0.53
fireblaster-yoshi   		29		13		0.45
superboomfan-kirby  		28		26		0.93
ld-fox              		26		16		0.62
shears-pika         		25		11		0.44
dext3r-pika         		24		8		0.33
superboomfan-pika   		22		18		0.82
isai-pika           		20		18		0.90
derek-pika          		17		10		0.59
isai-link           		17		11		0.65
prince-yoshi        		17		9		0.53
wario-pika          		17		13		0.76
lordnarwhal-falcon  		16		7		0.44
zero-falcon         		16		8		0.50
nothing-falcon      		16		7		0.44
jaimehr-falcon      		16		10		0.62
tacos-pika          		14		7		0.50
isai-fox            		13		8		0.62
tacos-falcon        		13		6

In [4]:

PLAYER_NAME = 'kerokeroppi'

print("Smash 64 Tournament Results")
print("\n\tData from following tournaments:")
for key in tournaments:
    print('\t\t' + key)

numchars = len(charnames)
numnames = len(playernames)
if skip_pools == 1:
    print("\tIncludes bracket games only")
if skip_pools != 1:
    print("\tIncludes bracket and pools")
    
print("\nplayer-character" + '\t\t' + "# games" + '\t\t' + "# wins" + '\t\t' + "% win")
inds = list(reversed(np.argsort(num2wins)))
total_games = 0
total_wins = 0
for j in range(len(playchar2num)):
    idx = inds[j]
    key = num2playchar[idx]
    ply_nm_len = len(PLAYER_NAME)
    if key[0:ply_nm_len] != PLAYER_NAME:
        continue
    val = int(playerchargames[key])
    valwins = int(playercharwins[key])
    total_games = total_games + val
    total_wins = total_wins + valwins
    print( key.ljust(20) + '\t\t' + str(val) + '\t\t' + str(valwins) + '\t\t' +  ("%.2f" % (valwins/val) ) )
print( '\n' + "TOTALS".ljust(20) + '\t\t' + str(total_games) + '\t\t' + str(total_wins) + '\t\t' +  ("%.2f" % (total_wins/total_games) ) )

Smash 64 Tournament Results

	Data from following tournaments:
		beast7
		goml
		genesis 3
		ssc
		snosa2
		pound
	Includes bracket and pools

player-character		# games		# wins		% win
kerokeroppi-pika    		57		33		0.58
kerokeroppi-falcon  		8		2		0.25
kerokeroppi-kirby   		8		5		0.62
kerokeroppi-fox     		3		2		0.67

TOTALS              		76		42		0.55
